In [1]:
from torchvision import transforms
import torch
import numpy as np
import pandas as pd
from pathlib import Path

import sys
sys.path.append('../')
import utils.image as im
import utils.json as json
import utils.path as path

from utils.VOCSegmentation import VOCSegmentation

root_path   = path.goback_from_current_dir(1)
json_path   = root_path + 'json\\'

In [2]:
data = VOCSegmentation(root = root_path,
                           year = '2012',
                           image_set = 'trainval',
                           download = False,
                           transform = transforms.ToTensor(),
                           target_transform = transforms.ToTensor(),
                           transforms = None,
                           target = 'Object')
data = iter(torch.utils.data.DataLoader(data,
                                        batch_size = 1,
                                        shuffle = False,
                                        num_workers = 0))

In [3]:
annotations = json.open_json(json_path + "voc-object-annotations-clean")
N = len(annotations)
annotations = iter(annotations.items())

classes = json.open_json(json_path + "voc-classes")

In [4]:
pixel = {c : 0 for c in classes}

In [5]:
for i in range(N):    
    img, sgm = next(data)
    sgm = torch.squeeze(sgm)

    sgm      = im.f1_to_f255(sgm.numpy())
    undef    = np.where(sgm == 255, 1, 0)
    
    nb_pixel = sgm.size - np.sum(undef)
    
    _, annots = next(annotations)

    for annot in annots:
        c, _, _ = annot
        
        pixel[c] = pixel[c] + nb_pixel
        
    if i % 100 == 0:
        print(i)

In [6]:
print(pixel)

{'aeroplane': 36104766, 'bicycle': 32324795, 'bird': 47869484, 'boat': 39348454, 'bottle': 59608857, 'bus': 40149580, 'car': 77223394, 'cat': 48294366, 'chair': 91333297, 'cow': 48001171, 'diningtable': 28497901, 'dog': 50318896, 'horse': 33961422, 'motorbike': 34257763, 'person': 286092724, 'pottedplant': 53625711, 'sheep': 54338101, 'sofa': 35565297, 'train': 32379798, 'tvmonitor': 34740846}


In [7]:
json.save_json(json_path + 'object_class_pixel' , list(map(int, list(pixel.values()))))

In [8]:
data = VOCSegmentation(root = root_path,
                           year = '2012',
                           image_set = 'trainval',
                           download = False,
                           transform = transforms.ToTensor(),
                           target_transform = transforms.ToTensor(),
                           transforms = None,
                           target = 'Class')
data = iter(torch.utils.data.DataLoader(data,
                                        batch_size = 1,
                                        shuffle = False,
                                        num_workers = 0))

annotations = json.open_json(json_path + "voc-class-annotations")
N = len(annotations)
annotations = iter(annotations.items())

In [9]:
pixel = {c : 0 for c in classes}

In [10]:
for i in range(N):    
    img, sgm = next(data)
    sgm = torch.squeeze(sgm)

    sgm      = im.f1_to_f255(sgm.numpy())
    undef    = np.where(sgm == 255, 1, 0)
    
    nb_pixel = sgm.size - np.sum(undef)
    
    _, annots = next(annotations)

    for c in annots:
        pixel[c] = pixel[c] + nb_pixel
        
    if i % 100 == 0:
        print(i)

In [11]:
print(pixel)

{'aeroplane': 29347825, 'bicycle': 23683797, 'bird': 36360396, 'boat': 25635827, 'bottle': 30660706, 'bus': 25950500, 'car': 42758578, 'cat': 42074813, 'chair': 45563155, 'cow': 23063593, 'diningtable': 26671665, 'dog': 41942977, 'horse': 24584431, 'motorbike': 26698617, 'person': 147216559, 'pottedplant': 27858032, 'sheep': 20955524, 'sofa': 31083202, 'train': 28577996, 'tvmonitor': 27583718}


In [12]:
json.save_json(json_path + 'class_class_pixel' , list(map(int, list(pixel.values()))))